In [2]:
import torch
from transformers import pipeline
from PIL import Image
import requests
import os
import pandas as pd
from tqdm import tqdm

# --- 1. Конфигурация ---
# Используем 7B-версию, так как 13B не поместится в VRAM
MODEL_ID = "llava-hf/llava-1.5-7b-hf"
IMG_DIR = "img"
RESULTS_FILE = "llava_pipeline_7b_results.csv"
PROMPT_TEMPLATE = "USER: <image>\n{prompt}\nASSISTANT:"
PROMPT_QUESTION = "Скажи, изображена ли на данном фото кошка породы \"Британская короткошерстная\", дай коротки ответ \"да\" или \"нет\". Опиши одним словом каждый признак, который убедил тебя в правильности или неправильности ответа. Если это не британская короткошерстная кошка, скажи какая это порода."

# --- 2. Загрузка модели через pipeline ---
pipe = None
print("Загрузка модели через pipeline... Это займет много времени и VRAM.")
try:
    pipe = pipeline(
        "image-to-text",
        model=MODEL_ID,
        torch_dtype=torch.float16, # Используем половинную точность для экономии памяти
        device_map="auto" # Автоматически использовать GPU
    )
    print("✅ Модель успешно загружена!")
except Exception as e:
    print(f"❌ Ошибка загрузки модели: {e}. Возможно, не хватает VRAM.")

# --- 3. Основной цикл обработки ---
if pipe:
    try:
        image_files = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.jpg', '.jpeg', '.png'))])
        results_data = []
        
        print(f"Начинаем обработку {len(image_files)} изображений...")
        for filename in tqdm(image_files, desc="LLaVA Pipeline 7B"):
            image_path = os.path.join(IMG_DIR, filename)
            
            try:
                # Открываем изображение
                image = Image.open(image_path)
                # Форматируем промпт правильно для LLaVA
                prompt = PROMPT_TEMPLATE.format(prompt=PROMPT_QUESTION)
                
                # Запускаем генерацию
                outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 50})
                
                # Извлекаем только ответ ассистента
                full_text = outputs[0]["generated_text"]
                result_text = full_text.split("ASSISTANT:")[1].strip()
                results_data.append({"filename": filename, "llava_7b_response": result_text})

            except Exception as e:
                results_data.append({"filename": filename, "llava_7b_response": f"Error: {e}"})

        df_results = pd.DataFrame(results_data)
        df_results.to_csv(RESULTS_FILE, index=False, encoding='utf-8')
        print(f"✅ Результаты сохранены в '{RESULTS_FILE}'.")
        display(df_results.head())

    except FileNotFoundError:
        print(f"❌ Ошибка: Директория '{IMG_DIR}' не найдена.")
else:
    print("❌ Основной процесс не был запущен из-за ошибки загрузки модели.")

Загрузка модели через pipeline... Это займет много времени и VRAM.


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

c:\Users\basal\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\basal\.cache\huggingface\hub\models--llava-hf--llava-1.5-7b-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
`torch_dtype` is deprecated! Use `dtype` instead!


❌ Ошибка загрузки модели: Could not load model llava-hf/llava-1.5-7b-hf with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForVision2Seq'>, <class 'transformers.models.llava.modeling_llava.LlavaForConditionalGeneration'>). See the original errors:

while loading with AutoModelForVision2Seq, an error is thrown:
Traceback (most recent call last):
  File "c:\Users\basal\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\pipelines\base.py", line 293, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "c:\Users\basal\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\models\auto\modeling_auto.py", line 2289, in from_pretrained
    return super().from_pretrained(pretrained_model_name_or_path, *model_args, **kwargs)
           ~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\basal\AppData\Local\Programs\Python\Python313\Lib\s

c:\Users\basal\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\models\auto\modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
